# Module 2: Frontier Models
## Lesson 11: The "Universal Client" Pattern (Gemini via OpenAI SDK)

### 📄 Overview
The "Chat Completions" API format (List of Messages -> JSON Response) has become the `de facto` standard for the AI industry. As a result, we don't need to learn a new library for every provider. We can simply point the standard `openai` client to a different URL.

### 🗝️ Key Concepts
* **Standardization:** Most Frontier Labs (Google, Anthropic, Groq, Mistral) now offer "OpenAI-Compatible Endpoints."
* **The `base_url` Parameter:** This is the magic switch. By default, the client points to `api.openai.com`. If we change this to Google's endpoint, the client sends the exact same JSON packet to Google instead.
* **Vendor Agnosticism:** This pattern allows engineers to swap models (e.g., move from GPT-4 to Gemini) by changing *configuration*, not *code logic*.

### 🛠️ Technical Implementation
To talk to Google Gemini using the OpenAI SDK, we need:
1.  **Google API Key:** Get this from [Google AI Studio](https://aistudio.google.com/).
2.  **Base URL:** Usually `https://generativelanguage.googleapis.com/v1beta/openai/` (Note: verify current endpoint in docs).

*Note: The instructor mentions "Gemini 2.5/3". In the code below, we use the current production model `gemini-1.5-flash`.*

In [ ]:
from openai import OpenAI
import os
from dotenv import load_dotenv

load_dotenv()

# 1. Retrieve the Google API Key (Add GOOGLE_API_KEY to your .env file first!)
google_key = os.getenv("GOOGLE_API_KEY")

if not google_key:
    print("❌ Error: Missing GOOGLE_API_KEY in .env")
else:
    print("✅ Found Google Key.")

# 2. Initialize the Client with a Custom Base URL
# This tells the library: "Don't go to OpenAI. Go here instead."
gemini_client = OpenAI(
    api_key=google_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

# 3. Make the Request (Identical syntax to previous lessons)
print("🌍 Calling Google Gemini...")

try:
    response = gemini_client.chat.completions.create(
        model="gemini-1.5-flash",  # Use a valid Gemini model name
        messages=[
            {"role": "user", "content": "Tell me a fun fact about flamingos."}
        ]
    )
    
    # 4. Extract Result
    print("\n--- Gemini Response ---")
    print(response.choices[0].message.content)

except Exception as e:
    print(f"❌ Error: {e}")

### 🧪 Lab Notes & Engineering Log

*The following experiments focus on Cross-Provider Compatibility.*

#### Experiment 1: The "Drop-in Replacement" Test
**Objective:** Verify if I can swap providers without breaking my `messages_for` function from Lesson 6.
**Test:**
1.  I used the *exact same* list of dictionaries `[{"role": "user" ...}]`.
2.  I sent it to Gemini.
3.  **Result:** Success. The schema is identical.
4.  **Implication:** I can write my entire application logic *once* and switch providers based on cost or availability just by changing a config file.

#### Experiment 2: Error Handling differences
**Observation:**
While the *request* is standard, the *errors* might differ.
* OpenAI returns `RateLimitError`.
* Google might return a 400 or 429 with different text.
* **Best Practice:** When building robust apps, we still need to catch generic `Exception` or map provider-specific errors if using advanced features.

#### Experiment 3: Finding the Endpoint
**Challenge:** Google's OpenAI-compatible endpoint URL changes occasionally as it moves from Beta to V1.
**Action:** I bookmarked the [Google AI Studio API Docs](https://ai.google.dev/) to ensure I have the correct `base_url`.